In [1]:
import numpy as np
import numpy.ma as ma
import xarray as xr
from matplotlib import pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as stats
import pandas as pd
from scipy.stats import norm, gamma
%matplotlib inline

In [2]:
fpath = '/glade/work/vcooper/atms_552_objanalysis/hw2/Data1b.txt'
file = open(fpath, 'r')
ds = np.loadtxt(file,dtype='float')
file.close()
ds

FileNotFoundError: [Errno 2] No such file or directory: '/glade/work/vcooper/atms_552_objanalysis/hw2/Data1b.txt'

In [ ]:
type(ds)

## First Distribution -- Normal

In [ ]:
# histogram to visualize pdf
plt.hist(ds[:,0], bins=20)

In [ ]:
np.histogram(ds[:,0], bins='auto')

In [ ]:
# Empirical Distribution
# create sorted series of data and plot Empirical CDF
# ECDF(x) = "number of samples <= x"/"number of samples"
cdfx1 = np.sort(ds[:,0]) # sorts the sample data in ascending order
totalp = np.arange(1,len(cdfx1)+1)/float(len(cdfx1)) # creates y-steps at 1/N, 2/N, ... N/N

plt.hist(cdfx1,density=True,bins=20, color='0.75', cumulative='True')
plt.plot(cdfx1,totalp, c='r', linewidth=3)
plt.title('Sample 1: Empirical CDF and Cumulative Standardized Histogram')

In [ ]:
# seaborn plot
sns.distplot(cdfx1)
plt.title('Sample 1: Empirical PDF and Standardized Histogram')

In [ ]:
fig = plt.subplots(ncols=1, nrows=1, figsize=(12,4))
param1 = norm.fit(cdfx1)
x = np.linspace(param1[0]-param1[1]*3.5,param1[0]+param1[1]*3.5,500)
pdf_fit = norm.pdf(x,param1[0],param1[1])

plt.plot(x, pdf_fit, linewidth=3, label="Fitted Normal PDF")

# seaborn plot
sns.distplot(cdfx1, color='r', kde_kws={"linewidth":3,"label":"Gaussian Kernel Density Estimate"})

plt.title('Sample 1: Fitted Normal Distribution PDF and Data-Driven Plots (Histogram and Kernel PDF)')
plt.show()

In [ ]:
print(param1)
print(np.mean(cdfx1),np.std(cdfx1))

## Second Distribution -- Gamma

In [ ]:
# Empirical Distribution
cdfx2 = np.sort(ds[:,1]) # sorts the sample data in ascending order
totalp = np.arange(1,len(cdfx1)+1)/float(len(cdfx2)) # creates y-steps at 1/N, 2/N, ... N/N

plt.hist(cdfx2,density=True,bins=20, color='0.75', cumulative='True')
plt.plot(cdfx2,totalp,c='r',linewidth=3)
plt.title('Sample 2: Empirical CDF and Cumulative Standardized Histogram')

In [ ]:
# seaborn plot
sns.distplot(cdfx2)
plt.title('Sample 2: Empirical PDF and Standardized Histogram')

In [ ]:
# Gamma fit for sample 2
param2 = gamma.fit(cdfx2)
param2 # alpha, location, beta

In [ ]:
fig = plt.subplots(ncols=1, nrows=1, figsize=(12,4))
x = np.linspace(0, max(cdfx2)+5,500)
plt.plot(x, gamma.pdf(x, param2[0], param2[1], param2[2]), linewidth=3, label="Fitted Gamma PDF")

# seaborn plot
sns.distplot(cdfx2, color='r', kde_kws={"linewidth":3,"label":"Gaussian Kernel Density Estimate"})

plt.title('Sample 2: Gamma Distribution PDF and Histogram of Data')
plt.show()

In [ ]:
print(param2)
print(np.mean(cdfx2),np.std(cdfx2))

## Pandas dataframe for summary statistics

In [ ]:
df1 = pd.DataFrame(cdfx1)
df2 = pd.DataFrame(cdfx2)

In [ ]:
df1.describe()

In [ ]:
# d = {'Stat':['Mean','Median','Mode','StdDev','Skew','Kurt','K-coef']}

In [ ]:
d = {'Stat':['Mean','Median','Mode','StdDev','Skew','Kurt','K-coef']}

In [ ]:
# appears that scipy uses sample sd, numpy uses population sd by default
print(stats.describe(cdfx1)[3]**0.5,(stats.describe(cdfx1)[3]**0.5)*((499/500)**0.5), np.std(cdfx1))

In [ ]:
dsT = np.transpose(ds) # transpose for enumerating
statnames = ['Mean','Median','Mode','PopulationSD','SampleSD','Skew','Kurt(4th moment)','K-coef','Excess K-coef']
temp = np.zeros(len(statnames))
stats_vals = np.array([temp,temp]) # empty to  hold statistics

for i,samp in enumerate(dsT):
    stats_vals[i,0] = np.mean(samp)
    stats_vals[i,1] = np.median(samp)
    stats_vals[i,2] = stats.mode(samp)[0].mean() # troubleshoot for mode array
    stats_vals[i,3] = np.std(samp)
    stats_vals[i,4] = np.std(samp,ddof=1) # adjusts std calc to divide by N-1 for sample sd
    stats_vals[i,5] = stats.skew(samp)
    stats_vals[i,6] = stats.moment(samp,moment=4)
    stats_vals[i,7] = stats.kurtosis(samp)+3
    stats_vals[i,8] = stats.kurtosis(samp) # convention is coeff of kurtosis
    
stats_vals

In [ ]:
# scipy descriptive stats
stats.describe(cdfx1)

In [ ]:
stats.describe(cdfx2)

In [ ]:
# Descriptive Statistics Table
table = pd.DataFrame(stats_vals,columns=statnames)
table

In [ ]:
# trying to figure out the subtleties
print(stats.moment(cdfx1,moment=3)) # skewness biased from moment
print(stats.skew(cdfx1)) # skewness coef
print(stats.skew(cdfx1,bias=False)) # skewness biased
print(stats.moment(cdfx1,moment=4)) # TBD why kurtosis is different from 4th moment
print(stats.kurtosis(cdfx1,fisher=True))
print(stats.kurtosis(cdfx1,fisher=False))
print(stats.moment(cdfx1,moment=2)**0.5)

# Dennis definition -- "Coefficient of Skew/Kurt" is the 'standardized' S/K and is the default output for scipy
# The pure "moment" calculations are the raw Skew/Kurt

In [ ]:
print(stats.moment(cdfx1,moment=3))
print(stats.moment(cdfx2,moment=3))
print(6/param2[0])

## Difference between data sets

In [ ]:
stats.ttest_ind(cdfx1,cdfx2,equal_var=False)

In [ ]:
tstat, p = stats.ttest_ind(cdfx1,cdfx2,equal_var=True) # default
tstat, p, 1-p

In [ ]:
help(table)

In [ ]:
## manual calculation of t-value suggests that formula in Hartmann book needs corrected
N = len(ds)
sd_hat = (((table['PopulationSD'][0]**2)*N+(table['PopulationSD'][1]**2)*N)/(N+N-2))**(0.5)
t = (table['Mean'][0]-table['Mean'][1])/(sd_hat*(1/N+1/N)**(0.5))
print(t, sd_hat)

In [ ]:
## manual calculation of t-value suggests that formula in Hartmann book needs corrected
N = len(ds)
sd_hat = (((table['SampleSD'][0]**2)*(N-1)+(table['SampleSD'][1]**2)*(N-1))/(N+N-2))**(0.5)
std_error = (sd_hat*(1/N+1/N)**(0.5))
t = (table['Mean'][0]-table['Mean'][1])/std_error
print(t, sd_hat)

In [ ]:
tc = stats.t.ppf(0.975, N + N - 2) # critical value for two tailed test
tc

In [ ]:
## Confidence interval calculation for t-test
# (x1-x2) +/- tc*SE where SE = pooled sd * sqroot(1/N1 + 1/N2)

mean_diff = (table['Mean'][0]-table['Mean'][1])
low = mean_diff - std_error*tc
high = mean_diff + std_error*tc
print('lower bound, difference of means, upper bound')
print(low,mean_diff,high)

In [ ]:
## Wilcoxon Rank Sum Test
z, pval = stats.ranksums(cdfx1,cdfx2)
z, pval, 1-pval

In [ ]:
## Wilcoxon Conf Int -- Ignore, this is useless
N1 = len(cdfx1)
N2 = len(cdfx2)
mu = N1*N2/2
sd = (N1*N2*(N1+N2+1)/12)**0.5
mu, sd

In [ ]:
# confirms that u stat from rank sum is a standard z score
zc = stats.norm.ppf(1-0.2042/2)
zc

In [ ]:
help(stats.ranksums)

In [ ]:
np.exp(1)

## Radiation HW1

In [ ]:
# Planck Function Constants
c = 2.9979e+8
h = 6.626e-34
k = 1.3806e-23
sigma = 5.66961e-8
Ts = 5800 # photosphere temp per Liou book
Te = 255 # earth-atmos eq temp per Liou book
xs = np.linspace(0,5,1000) # 0 to 5 microns for sun x-axis
xe = np.linspace(0,100,1000) # 0 to 40 microns for earth x-axis

In [ ]:
def planck(T,wavelength): # wavelength in microns
    c = 2.9979e+8
    h = 6.626e-34
    k = 1.3806e-23
    wavelength = wavelength*(10**(-6)) # convert microns to meters
    bb_radiance = (2*h*(c**2))/(wavelength**5*(np.exp(h*c/(k*wavelength*T))-1))
    bb_radiance = bb_radiance/(10**6) # convert to microns
    return(bb_radiance)

In [ ]:
def rayj(T,wavelength): # wavelength in microns
    c = 2.9979e+8
    h = 6.626e-34
    k = 1.3806e-23
    wavelength = wavelength*(10**(-6)) # convert microns to meters
    bb_radiance = (2*c*k*T)/(wavelength**4)
    bb_radiance = bb_radiance/(10**6) # convert to microns
    return(bb_radiance)

In [ ]:
def wien(T,wavelength): # wavelength in microns
    c = 2.9979e+8
    h = 6.626e-34
    k = 1.3806e-23
    wavelength = wavelength*(10**(-6)) # convert microns to meters
    bb_radiance = (2*h*(c**2))/(wavelength**5*(np.exp(h*c/(k*wavelength*T))))
    bb_radiance = bb_radiance/(10**6) # convert to microns
    return(bb_radiance)

In [ ]:
planck(Ts,0.5)/10**7

In [ ]:
# Solar from 5 to infinity from https://www.spectralcalc.com/blackbody_calculator/blackbody.php
105787/(sigma/np.pi*Ts**4)

In [ ]:
# Earth from 0 to 5 microns from https://www.spectralcalc.com/blackbody_calculator/blackbody.php
0.279531/(sigma/np.pi*Te**4)

In [ ]:
# Sun
plt.plot(xs,planck(Ts,xs),c='r',linewidth=3)
plt.plot(xs,wien(Ts,xs),c='g')
plt.plot(xs,rayj(Ts,xs))
plt.ylim(0,3e+7)

In [ ]:
# Sun
plt.plot(xs,planck(Ts,xs),c='r',linewidth=3)
plt.plot(xs,wien(Ts,xs),c='g')
plt.plot(xs,rayj(Ts,xs))
plt.ylim(0,0.1e+7)
plt.xlim(0,6)

In [3]:
# Earth
plt.plot(xe,planck(Te,xe),c='r',linewidth=3)
plt.plot(xe,wien(Te,xe),c='g')
plt.plot(xe,rayj(Te,xe))
plt.ylim(0,4.5)

NameError: name 'xe' is not defined

(6,)

In [86]:
w = np.asarray([1, .999, .99, .9, .7, 0])
g = np.asarray([0, .75, .85])
u = np.asarray([0.9063, 2/3, 0.2588])

In [92]:
def albedo(u,w,g):
    Ftop = 1 # this actually does not get used because it cancels out
    Ibar = 4
    tau = 0
    B = Ftop*(3*w*g*(1-w)*u+w/u)
    A = 3*(1-w)*(1-w*g)
    #C = (4*np.pi*(1-w)*Ibar-w*Ftop+B*u*(1-A*u**2)**(-1))/np.sqrt(A)
    C = (2*B*u**2/(1-A*u**2)*(-1/2/u-1+w)+w*Ftop)/(2*(1-w)+np.sqrt(A))
    F = B*u**2*np.exp(-tau/u)/(1-A*u**2)+C*np.exp(-np.sqrt(A)*tau)
    albedo = F/(u*Ftop)
    return(albedo)

In [93]:
ans = np.zeros(len(w)*len(g)*len(u)).reshape(len(w),len(g),len(u))

for i,wval in enumerate(w):
    for j,gval in enumerate(g):
        ans[i,j,:] = albedo(u,wval,gval)

/home/disk/atmos/vcooper/miniconda3/envs/beauchard/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
/home/disk/atmos/vcooper/miniconda3/envs/beauchard/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [94]:
for i,wval in enumerate(w):
    for j,gval in enumerate(g):
        print(wval,gval,ans[i,j,:])

1.0 0.0 [-inf  nan  nan]
1.0 0.75 [-inf  nan  nan]
1.0 0.85 [-inf  nan  nan]
0.999 0.0 [0.91822593 0.92985331 0.95033569]
0.999 0.75 [0.84097296 0.86437633 0.90491233]
0.999 0.85 [0.79855208 0.82842909 0.8799777 ]
0.99 0.0 [0.76710183 0.79564505 0.8494416 ]
0.99 0.75 [0.57367467 0.63182395 0.73630388]
0.99 0.85 [0.4802093  0.5528124  0.68180689]
0.9 0.0 [0.44056975 0.48292846 0.57741945]
0.9 0.75 [0.14835255 0.23594905 0.41293235]
0.9 0.85 [0.05362513 0.15689589 0.36094352]
0.7 0.0 [0.23056447 0.26267293 0.3442756 ]
0.7 0.75 [0.01475227 0.07925287 0.22878687]
0.7 0.85 [-0.03367015  0.03886716  0.20407061]
0.0 0.0 [0. 0. 0.]
0.0 0.75 [0. 0. 0.]
0.0 0.85 [0. 0. 0.]


In [95]:
for i,wval in enumerate(w):
    print('W =',wval)
    print(pd.DataFrame(ans[i,:,:],columns=[g[0],g[1],g[2]],index=[u[0],u[1],u[2]]),'\n')
    #for j,gval in enumerate(g):
        #print(wval,gval,ans[i,j,:])


W = 1.0
          0.00  0.75  0.85
0.906300  -inf   NaN   NaN
0.666667  -inf   NaN   NaN
0.258800  -inf   NaN   NaN 

W = 0.999
              0.00      0.75      0.85
0.906300  0.918226  0.929853  0.950336
0.666667  0.840973  0.864376  0.904912
0.258800  0.798552  0.828429  0.879978 

W = 0.99
              0.00      0.75      0.85
0.906300  0.767102  0.795645  0.849442
0.666667  0.573675  0.631824  0.736304
0.258800  0.480209  0.552812  0.681807 

W = 0.9
              0.00      0.75      0.85
0.906300  0.440570  0.482928  0.577419
0.666667  0.148353  0.235949  0.412932
0.258800  0.053625  0.156896  0.360944 

W = 0.7
              0.00      0.75      0.85
0.906300  0.230564  0.262673  0.344276
0.666667  0.014752  0.079253  0.228787
0.258800 -0.033670  0.038867  0.204071 

W = 0.0
          0.00  0.75  0.85
0.906300   0.0   0.0   0.0
0.666667   0.0   0.0   0.0
0.258800   0.0   0.0   0.0 



In [96]:
ans[1,:,:].shape

(3, 3)

In [ ]:
pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])

In [67]:
test.shape

(3, 3)

In [49]:
alb[0][:]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [50]:
len(albedo)

TypeError: object of type 'function' has no len()